In [1]:
import time
time.gmtime()

time.struct_time(tm_year=2021, tm_mon=8, tm_mday=22, tm_hour=12, tm_min=20, tm_sec=4, tm_wday=6, tm_yday=234, tm_isdst=0)

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [3]:
import pandas as pd

from _code import DeepLogModel
from _code import generator
from _code import trie
from _code import seed

In [4]:
seed.seed(0)

In [5]:
vocabulary_size = 10
vocabulary = generator.make_vocabulary(vocabulary_size=vocabulary_size)
patterns = generator.generate_patterns(num_patterns=15, vocabulary=vocabulary, min_pattern_size=3, max_pattern_size=7)

p_ = [(v,) + p for v,p in zip(vocabulary, patterns[:vocabulary_size])]
patterns = p_ + patterns[vocabulary_size:]

trie_g = trie.calc_g_value(patterns)
trie_h = trie.calc_h_value(patterns)
print("trie-g:", trie_g)
print("trie-h:", trie_h)

trie-g: 10
trie-h: 3


In [6]:
text_train, marks_train = generator.generate_text(patterns, text_size=50000, anomaly_ratio=0.00, vocabulary=vocabulary) 

In [7]:
deep_log_model = DeepLogModel.DeepLogModel(h=trie_h+1, n=vocabulary_size, vocabulary=vocabulary)
deep_log_model.build(num_lstm_layers=2, lstm_size=64)
deep_log_model.fit(text_train,epochs=1)

In [8]:
tests_legit = generator.generate_tests(patterns, vocabulary, n=1, text_size = 100, anomaly_ratio=0.00)
text_test_legit, marks_test_legit, anomaly = tests_legit[0]

tests_anomaly = generator.generate_tests(patterns, vocabulary, n=1, text_size = 100, anomaly_ratio=0.05)
text_test_anomaly, marks_test_anomaly, anomaly = tests_anomaly[0]

In [9]:
entries = list()
for g in range(0, vocabulary_size+1):
    res_legit = deep_log_model.monitor_session(text_test_legit, marks_test_legit,g=g)
    res_anomaly = deep_log_model.monitor_session(text_test_anomaly, marks_test_anomaly,g=g)
    entry = (g,res_legit, res_anomaly)
    entries.append(entry)
df = pd.DataFrame(entries, columns = ["g", "without_anomaly", "with_anomaly"])
df

,g,without_anomaly,with_anomaly
0,0,FP,TP
1,1,FP,TP
2,2,FP,TP
3,3,FP,TP
4,4,FP,TP
5,5,FP,TP
6,6,FP,TP
7,7,FP,TP
8,8,FP,TP
9,9,FP,TP


In [10]:
df["total"] = df[["without_anomaly", "with_anomaly"]].min(axis=1)
def color(x):
    if x in {"FP","FN"}:
        return "background-color: red"
    elif x in {"TP","TN"}:
        return "background-color: green"
    else:
        raise ValueError()
        
df.style.applymap(color, subset = ["total"])

,g,without_anomaly,with_anomaly,total
0,0,FP,TP,FP
1,1,FP,TP,FP
2,2,FP,TP,FP
3,3,FP,TP,FP
4,4,FP,TP,FP
5,5,FP,TP,FP
6,6,FP,TP,FP
7,7,FP,TP,FP
8,8,FP,TP,FP
9,9,FP,TP,FP
